In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
# define raw data directory
data_path = './raw_data/'

# the car company list u want to download
car_com_lst = ['TSLA',
                'GM',
                'F']

In [3]:
# car_com_lst = ['BZ=F'] # to get oil future price

In [4]:
for carCom in car_com_lst:
    df = yf.download(carCom, 
                     start='2021-01-01', 
                     end='2023-03-25', 
                     # period = '5d',
                     progress=False)
    
    df.to_csv(data_path + carCom + "_20210101_to_20230325.csv")

In [6]:
TSLA_df = pd.read_csv(data_path + 'TSLA_20210101_to_20230325.csv', parse_dates=['Date'])
GM_df = pd.read_csv(data_path + 'GM_20210101_to_20230325.csv', parse_dates=['Date'])
F_df = pd.read_csv(data_path + 'F_20210101_to_20230325.csv', parse_dates=['Date'])
DJAHD_df = pd.read_csv(data_path + 'Dow Jones Automobiles Historical Data.csv', parse_dates=['Date'])
Brent_df = pd.read_csv(data_path + 'Europe_Brent_Spot_Price_FOB_modified.csv', parse_dates=['Day'])

In [7]:
DJAHD_df['Price'] = DJAHD_df['Price'].str.replace(',', '').astype(float)
DJAHD_df['Open'] = DJAHD_df['Open'].str.replace(',', '').astype(float)
DJAHD_df['High'] = DJAHD_df['High'].str.replace(',', '').astype(float)
DJAHD_df['Low'] = DJAHD_df['Low'].str.replace(',', '').astype(float)
DJAHD_df['Vol.'] = DJAHD_df['Vol.'].str.replace(',', '')
DJAHD_df['Vol.'] = DJAHD_df['Vol.'].str.replace('M', '').astype(float)
DJAHD_df['Vol.'] = DJAHD_df['Vol.']*1000000
DJAHD_df['Vol.'] = DJAHD_df['Vol.'].astype(int)
DJAHD_df['Change %'] = DJAHD_df['Change %'].str.replace('%', '').astype(float)

In [8]:
final_df = TSLA_df.set_index('Date')
final_df = final_df.join(GM_df.set_index('Date'), how='left', lsuffix='_TSLA', rsuffix='_GM')
final_df = final_df.join(F_df.set_index('Date'), how='left')
final_df = final_df.rename(columns={"Close": "Close_F", 
                                    "Open": "Open_F", 
                                    "High": "High_F", 
                                    "Low": "Low_F", 
                                    "Adj Close": "Adj Close_F",
                                    "Volume": "Volume_F"})

In [9]:
final_df = final_df.join(DJAHD_df.set_index('Date'), how='left')
final_df = final_df.rename(columns={"Price":"Price_DJAHD",
                                    "Open":"Open_DJAHD",
                                    "High":"High_DJAHD",
                                    "Low":"Low_DJAHD",
                                    "Vol.":"Vol._DJAHD",
                                    "Change %":"Change %_DJAHD"})

In [10]:
final_df = final_df.join(Brent_df.set_index('Day'), how='left')

In [11]:
final_df.dtypes

Open_TSLA                                          float64
High_TSLA                                          float64
Low_TSLA                                           float64
Close_TSLA                                         float64
Adj Close_TSLA                                     float64
Volume_TSLA                                          int64
Open_GM                                            float64
High_GM                                            float64
Low_GM                                             float64
Close_GM                                           float64
Adj Close_GM                                       float64
Volume_GM                                            int64
Open_F                                             float64
High_F                                             float64
Low_F                                              float64
Close_F                                            float64
Adj Close_F                                        float

In [12]:
final_df[final_df.isna().any(axis=1)]

,Open_TSLA,High_TSLA,Low_TSLA,Close_TSLA,Adj Close_TSLA,Volume_TSLA,Open_GM,High_GM,Low_GM,Close_GM,...,Close_F,Adj Close_F,Volume_F,Price_DJAHD,Open_DJAHD,High_DJAHD,Low_DJAHD,Vol._DJAHD,Change %_DJAHD,Europe Brent Spot Price FOB Dollars per Barrel
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-05,235.903336,236.053329,228.233337,230.350006,230.350006,125528400,59.000000,61.189999,58.750000,61.040001,...,12.700000,11.511686,82332700,901.49,910.12,916.34,892.86,149290000.0,4.54,NaN
2021-05-03,234.600006,235.333328,226.833328,228.300003,228.300003,81129300,57.599998,58.340000,56.709999,57.150002,...,11.630000,10.541804,61987300,884.35,904.24,906.80,879.24,100670000.0,-2.77,NaN
2021-08-30,238.240005,243.666672,237.576660,243.636673,243.636673,55812600,49.950001,49.950001,48.950001,49.169998,...,13.050000,11.828938,42720100,924.20,907.27,924.60,907.27,75290000.0,1.92,NaN
2021-12-27,357.890015,372.333344,356.906677,364.646667,364.646667,71145900,56.889999,57.750000,56.529999,57.430000,...,20.799999,18.948719,59651800,1346.90,1320.55,1369.33,1318.59,156190000.0,2.52,NaN
2021-12-28,369.829987,373.000000,359.473328,362.823334,362.823334,60324000,57.400002,58.119999,57.009998,57.110001,...,20.760000,18.912281,53020500,1338.92,1361.46,1368.97,1331.33,121980000.0,-0.59,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-20,178.080002,186.440002,176.350006,183.250000,183.250000,129684400,33.450001,34.250000,33.270000,33.619999,...,11.180000,11.180000,74975100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-21,188.279999,198.000000,188.039993,197.580002,197.580002,153391400,34.630001,35.320000,34.459999,35.099998,...,11.720000,11.720000,74794900,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-22,199.300003,200.660004,190.949997,191.149994,191.149994,150376400,35.130001,35.320000,34.029999,34.049999,...,11.480000,11.480000,69729100,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
final_df = final_df.fillna(method='ffill')
final_df[final_df.isna().any(axis=1)]

,Open_TSLA,High_TSLA,Low_TSLA,Close_TSLA,Adj Close_TSLA,Volume_TSLA,Open_GM,High_GM,Low_GM,Close_GM,...,Close_F,Adj Close_F,Volume_F,Price_DJAHD,Open_DJAHD,High_DJAHD,Low_DJAHD,Vol._DJAHD,Change %_DJAHD,Europe Brent Spot Price FOB Dollars per Barrel
Date,,,,,,,,,,,,,,,,,,,,,


In [16]:
# save the file
final_df.to_pickle("stock_price_all.pkl")

In [17]:
final_df

,Open_TSLA,High_TSLA,Low_TSLA,Close_TSLA,Adj Close_TSLA,Volume_TSLA,Open_GM,High_GM,Low_GM,Close_GM,...,Close_F,Adj Close_F,Volume_F,Price_DJAHD,Open_DJAHD,High_DJAHD,Low_DJAHD,Vol._DJAHD,Change %_DJAHD,Europe Brent Spot Price FOB Dollars per Barrel
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,239.820007,248.163330,239.063339,243.256668,243.256668,145914600,41.950001,42.119999,40.139999,40.509998,...,8.52,7.722802,85043100,882.86,875.67,900.12,870.29,148370000.0,2.50,50.37
2021-01-05,241.220001,246.946671,239.733337,245.036667,245.036667,96735600,40.060001,41.840000,40.040001,41.660000,...,8.65,7.840636,70127800,891.16,876.49,896.46,872.02,118400000.0,0.94,53.16
2021-01-06,252.830002,258.000000,249.699997,251.993332,251.993332,134100000,41.919998,43.849998,41.860001,42.980000,...,8.84,8.012860,72590200,916.56,915.63,937.07,908.37,140230000.0,2.85,53.80
2021-01-07,259.209991,272.329987,258.399994,272.013336,272.013336,154496700,43.980000,43.980000,42.959999,43.320000,...,9.06,8.212275,77117100,980.89,940.51,981.91,939.29,141230000.0,7.02,53.70
2021-01-08,285.333344,294.829987,279.463318,293.339996,293.339996,225166500,43.310001,43.490002,42.320000,43.060001,...,9.00,8.157888,59162200,1046.75,1022.40,1051.36,1003.05,152540000.0,6.71,55.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-20,178.080002,186.440002,176.350006,183.250000,183.250000,129684400,33.450001,34.250000,33.270000,33.619999,...,11.18,11.180000,74975100,1311.02,1322.00,1334.84,1309.05,100650000.0,-0.88,85.86
2023-03-21,188.279999,198.000000,188.039993,197.580002,197.580002,153391400,34.630001,35.320000,34.459999,35.099998,...,11.72,11.720000,74794900,1311.02,1322.00,1334.84,1309.05,100650000.0,-0.88,85.86
2023-03-22,199.300003,200.660004,190.949997,191.149994,191.149994,150376400,35.130001,35.320000,34.029999,34.049999,...,11.48,11.480000,69729100,1311.02,1322.00,1334.84,1309.05,100650000.0,-0.88,85.86
